In [1]:
import cv2
import numpy as np

In [2]:
cap = cv2.VideoCapture(0)

In [3]:
sharp_kernel = np.array([[0, -1, 0], 
                         [-1, 5, -1], 
                         [0, -1, 0]]) 

In [3]:
acc_image = np.zeros((480, 640, 3))

while True:
    _, frame = cap.read()
    hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    low_blue = np.array([94, 40, 120])
    high_blue = np.array([126, 255, 255])
    blue_mask = cv2.inRange(hsv_frame, low_blue, high_blue)
    blue = cv2.bitwise_and(frame, frame, mask=blue_mask)
    blue = cv2.GaussianBlur(blue, (5,5), cv2.BORDER_DEFAULT)

    # Accumulating the highlight
    acc_image = ((blue + acc_image.astype(float) * 1) / 2).astype(np.uint8)

    # Converting from HSV to GRAYSCALE
    blue_gray = cv2.cvtColor(cv2.cvtColor(acc_image, cv2.COLOR_HSV2BGR), cv2.COLOR_BGR2GRAY)

    # Adaptive threshold to select the blue highlights
    th = cv2.adaptiveThreshold(blue_gray,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY,21,4)

    # Gaussian blur to smoothen the mask
    th = cv2.GaussianBlur(th,(5,5),cv2.BORDER_DEFAULT)

    #clipping_func = np.vectorize(lambda x: min(255, x * 3))

    # Adding the glow to the original frame
    glowing_blue = cv2.add(frame, acc_image.astype(np.uint8), th.astype(float))

    #frame_without_blue = cv2.addWeighted()
    cv2.imshow("Glowing blue", glowing_blue)
    #cv2.imshow("blue_gray", blue_gray)
    #cv2.imshow("Frame", th)
    #cv2.imshow("Added", )
    key = cv2.waitKey(1)
    if key == 27:
        cv2.destroyAllWindows()
        break;

In [9]:
cv2.destroyAllWindows()